# Algorithme des *k* plus proches voisins

L'objectif de ce TP est de mettre en œuvre une méthode, celle des $k$ plus proches voisins, qui permet à l’ordinateur de reconnaître automatiquement des chiffres manuscrits (pour lire des chèques, lire des codes postaux par exemple). ![enveloppe](lettres.jpg)

Les chiffres seront donnés sous forme d’images, puisqu’ils sont scannés par une machine avant d’être traités par l’ordinateur.

## 1. Distance entre deux images

### 1.1. Définition d’une distance particulière

Pour évaluer la différence entre deux images, on va définir une distance entre deux images. Les images que nous traiterons sont au format [PGM](https://fr.wikipedia.org/wiki/Portable_pixmap#Fichier_binaire_2), c'est à dire en niveau de gris.

* Pour tous les pixels de l'image on calcule la différence des valeurs d'intensité, dont on supprime le signe (abs)
* On somme toutes ces différences

1. Déterminer à la main la distance entre les deux images ci-dessous.

![comparaison](distance_eleve.png)

La distance vaut : (compléter) 2×(255−255)+(255−191)+3×(255−0)+(191−40)+(0−0)+(191−0)=1171

2. Écrire une fonction qui calcule la distance entre deux images. Une image sera représentée comme une liste d’entiers, valeurs de gris des pixels.

In [1]:
def distance (a, b):
    somme = 0
    for i in range (len(a)):
        somme += abs(a[i] - b [i]) 
    return somme

def distance(a,b):
    return sum([abs(a[i]-b[i]) for i in range(len(a))])

### 2.2. Exemple d'utilisation

Nous allons calculer la distance entre les images suivantes :

|ecrit1.pgm|ecrit2.pgm|ecrit3.pgm|
|------|------|-----|
|![chiffre5](ecrit1.png)|![chiffre5](ecrit2.png)|![chiffre5](ecrit3.png)|

On cherche à comparer la première image aux deux suivantes. Bien sûr, nous savons très bien dire quelle image lui ressemble le plus. Cependant notre but étant d’avoir un moyen automatique, nous devons vérifier que celui que nous utilisons fonctionne bien.

3. Déterminer les distances entre la première image avec chacune des deux autres, et déterminer alors laquelle est la plus proche de la première.

Pour cela, on utilise la librairie de manipulation d’images PIL.

**Utilisation de la librairie PIL :**
* ouvrir une image `Image.open(nom du fichier)`
* Récuperer les valeurs des pixels de notre image `monimage.getdata()`, il faut la transformer en liste avec la commande `list(...)`


In [2]:
from PIL import Image

ecrit1 = Image.open("ecrit1.pgm")
# met l'image dans un tableau qui est de dimension 784
img1 =list(ecrit1.getdata())

ecrit2 = Image.open("ecrit2.pgm")
# met l'image dans un tableau qui est de dimension 784
img2 =list(ecrit1.getdata())

ecrit3 = Image.open("ecrit3.pgm")
# met l'image dans un tableau qui est de dimension 784
img3 =list(ecrit1.getdata())

d12 = distance(img1,img2)
d13 = distance(img1,img3)

if d13<d12:
    print("ecrit1.pgm est plus proche de ecrit3.pgm que de ecrit2.pgm")
else:
    print("ecrit1.pgm est plus proche de ecrit2.pgm que de ecrit3.pgm")

ecrit1.pgm est plus proche de ecrit2.pgm que de ecrit3.pgm


### 2.3 les k plus proches voisins
Sur le dessin ci-dessous nous avons représenté un chiffre dont on cherche à l’identifier. Pour cela nous allons déterminer à l’aide de la méthode des k-NN (k plus proches voisins).
On cherche à déterminer les proches voisins du caractère entouré
![trouver](trouver.png)
1. Déterminer les 3-plus proches voisins.
2. Déterminer les 6-plus proches voisins.
3. Que peut-on en déduire ?

## 2. Parcours d’un jeu de données

### 1. Principe

Avant même d'implanter l'algorithme des *k* plus proches voisins, nous avons besoin d'exemples qui seront traités par celui-ci. Aussi, commençons par lire un jeu de données.

Il s'agit du jeu de données, très utilisé en apprentissage automatique, appelé MNIST. Il est constitué d'un ensemble de 70000 images, de format 28x28 pixels, en noir et blanc annoté du chiffre correspondant (entre 0 et 9). Ce jeu utilise des données réelles qui ont déjà été pré-traitées pour être plus facilement utilisables par notre algorithme.

![Un extrait du type d'images que l'on trouve dans le dataset MNIST](extrait_MNIST.png)

Nous prendrons de cette base de données un extrait de 5000 données uniquement, afin de limiter les temps de calcul de notre algorithme.

Il nous faut obtenir deux listes, **data** et **target**.

* **data** contient les images sous forme d'une liste de 28*28 = 784 entiers compris entre 0 et 255 correspondant aux différentes nuances de gris (255 étant blanc et 0 noir)
* **target** contient les chiffres (de type int) correspondant à l'image

Tout d’abord on récupère la base de donnée complète (cela peut prendre beaucoup de temps) :

In [3]:
from sklearn.datasets import fetch_openml # Note : il faut avoir installé la librairie sklearn pour python

mnist = fetch_openml('mnist_784', version=1)

Ensuite on en prend l’extrait que l’on met sous la forme qui nous convient (on aura besoin de 1000 autres images plus tard, d’où les 6000) :

In [4]:
import numpy as np

sample = np.random.randint(70000, size=6000)

data = [mnist.data[i].astype(int).tolist() for i in sample[:5000]]

target = [int(mnist.target[i]) for i in sample[:5000]]

4. Afficher la liste correspondant à l’image d’indice 23 du jeu de données. À quel chiffre corrrespond cette image ?

In [5]:
print(data[23])

print('Le chiffre representé est :', target[23])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 55, 236, 107, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 112, 216, 51, 0, 0, 0, 0, 16, 232, 252, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 148, 252, 126, 0, 0, 0, 0, 22, 252, 252, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 174, 252, 126, 0, 0, 0, 0, 22, 252, 252, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 253, 252, 126, 0, 0, 0, 0, 110, 252, 252, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 107, 254, 253, 109, 0, 0, 0, 0, 84, 253, 236, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

5. Dans la cellule ci-dessous, prendre deux élèments quelconques au choix de notre jeu de données, nommées image1 et image2. calculer alors la distance entre ces deux images et indiquer le chiffre que chacune représente.

In [6]:
i1=23
i2=65
image1 = data[i1]
image2 = data[i2]
print(distance(image1,image2))
print(target[i1])
print(target[i2])

26978
4
8


## 3. Déterminer les *k* plus proches voisins d'un point

### 3.1 Le plus proche voisin

6. Écrire une fonction lePlusProcheVoisin qui prend en paramètre une image (toujours une liste de 784 entiers compris entre 0 et 255) et renvoie l'indice dans data du plus proche voisin.

In [7]:
def lePlusProcheVoisin (image):
    lePlusPres = 0
    distanceMin = float("inf")
    for i in range(len(data)):
        di = distance (image, data[i])
        if di != 0 and di < distanceMin:
            lePlusPres = i
            distanceMin = di
    return (lePlusPres)

7. Appliquer cet algorithme à image1 et image2, et vérifier si l’algorithme a trouvé un voisin qui représente le même chiffre.

In [8]:
indice1 = lePlusProcheVoisin(image1)
print(target[indice1])
print(target[i1])

indice2 = lePlusProcheVoisin(image2)
print(target[indice2])
print(target[i2])

4
4
8
8


On remarque éventuellement que ce n’est pas forcément parfait, le résultat n’étant pas toujours celui attendu.

### 3.2 Les *k* plus proches voisins

8. Écrire une fonction lesKplusProchesVoisins qui prend en paramètre une image et une valeur de k et renvoie la liste des indices dans data des k plus proches voisins. Quand vous prenez k = 1, cette fonction doit renvoyer le même résultat que la précédente, mis à part le fait que lePlusProcheVoisin renvoie une valeur numérique alors que lesKplusProchesVoisins renvoie une liste d'un élément.

In [9]:
def lesKplusProchesVoisins (image, k):
    listeDesDistances = []
    for img in data:
        listeDesDistances.append (distance (image, img))
    Kppv = []
    for i in range (k):
        p = float ("inf")
        for j in range (len (data)):
            if listeDesDistances [j] != 0 and listeDesDistances [j] < p and j not in Kppv:
                p = listeDesDistances [j]
                indice = j
        Kppv. append (indice)
    return (Kppv)


9. Exécuter l’algorithme avec image1 et k=5, puis donner la liste des chiffres associés aux images dont l’algorithme donne les indices dans notre base.

In [10]:
Kppv = lesKplusProchesVoisins (image1, 5)
print([target[indice] for indice in Kppv])

[4, 4, 4, 4, 4]


## 4. Prédire l'étiquette d'une donnée

10. Écrire une fonction *predire* qui prend en paramètres une image dans le même format que celles de data et un entier *k* et retourne le chiffre qui est prédit, c'est à dire le chiffre qui est supposé être représenté sur l'image. On décide du chiffre représenté sur l'image en appliquant un choix à la majorité, à savoir le chiffre qui apparaît majoritairement sur les *k* plus proches voisins. Donner alors la valeur prédite pour image1, toujours pour k=5.

In [11]:
def predire (l,k):
    Kppv = lesKplusProchesVoisins(l,k)
    decomptes = [0]*10
    for indice in Kppv:
        decomptes[target[indice]] += 1
    plusGrandDecompte = decomptes [0]
    for i in range (1,10):
        if decomptes [i] > plusGrandDecompte:
                plusGrandDecompte = decomptes [i]
                indice = i
    return indice

print(predire(image1,5))

4


## 5. Optimisation

Prenons maintenant un jeu d’images à tester, qui sont en dehors de la base des 5000 images traitées par l’algorithme.

In [12]:
data_test = [mnist.data[i].astype(int).tolist() for i in sample[5000:]]

target_test = [int(mnist.target[i]) for i in sample[5000:]]

Notre but est de calculer le taux d’erreur des prédictions, puis de chercher la meilleure valeur de *k* à choisir pour diminuer celui-ci.

11. Écrire une fonction qui calcule le taux d’erreur sur les prédictions portant sur le jeu d’images à tester. Exécuter ensuite cette fonction pour une valeur de *k* choisie entre 2 et 7.

In [13]:
def taux_erreur(k):
    '''
    calcule le taux d'erreur avec la valeur k pour les k plus proches voisins
    '''
    t=0
    n=100#len(target_test)
    for i in range(n):
        if predire(data_test[i],k)!=target_test[i]:
            t+=1
    return t/n

print(taux_erreur(5))

0.2


12. Écrire une fonction optimisation(n) qui détermine la valeur de *k*, entre 1 et n, qui minimise le taux d’erreur, puis donner le résultat de cette fonction avec n=7.

In [ ]:
def optimisation(n):
    '''
    détermine quelle valeur de k donne la meilleure prédiction, avec k entre 1 et n
    '''
    liste_taux=[taux_erreur(k) for k in range(1,n+1)]
    return liste_taux.index(min(liste_taux))+1

print(optimisation(7))